In [1]:
import pandas as pd
import folium
import ast
from datetime import datetime, timedelta

In [2]:
# Load the dataset
file_path = '/content/police-department-incidents.csv'
data = pd.read_csv(file_path)

<ipython-input-2-b73ff2919208>:3: DtypeWarning: Columns (1,2,3,4,5,6,7,8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


In [19]:
# Convert 'Date' column to datetime and filter for the latest 7 days
data['Date'] = pd.to_datetime(data['Date'])
max_date = data['Date'].max()
last_7_days = data[data['Date'] >= (max_date - timedelta(days=1))]

In [20]:
last_7_days

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,180362289.0,VEHICLE THEFT,STOLEN MOTORCYCLE,Tuesday,2018-05-15,10:30,SOUTHERN,NONE,700 Block of TEHAMA ST,-122.411912,37.775207,"{'type': 'Point', 'coordinates': [-122.4119120...",1.803620e+13
1,180360948.0,NON-CRIMINAL,"AIDED CASE, MENTAL DISTURBED",Tuesday,2018-05-15,04:14,SOUTHERN,NONE,MARKET ST / SOUTH VAN NESS AV,-122.419258,37.775146,"{'type': 'Point', 'coordinates': [-122.4192578...",1.803610e+13
2,180360879.0,OTHER OFFENSES,PAROLE VIOLATION,Tuesday,2018-05-15,02:01,MISSION,"ARREST, BOOKED",CAPP ST / 21ST ST,-122.417813,37.757101,"{'type': 'Point', 'coordinates': [-122.4178125...",1.803610e+13
3,180360879.0,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Tuesday,2018-05-15,02:01,MISSION,"ARREST, BOOKED",CAPP ST / 21ST ST,-122.417813,37.757101,"{'type': 'Point', 'coordinates': [-122.4178125...",1.803610e+13
4,180360879.0,OTHER OFFENSES,TRAFFIC VIOLATION,Tuesday,2018-05-15,02:01,MISSION,"ARREST, BOOKED",CAPP ST / 21ST ST,-122.417813,37.757101,"{'type': 'Point', 'coordinates': [-122.4178125...",1.803610e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,180358012.0,OTHER OFFENSES,PROBATION VIOLATION,Monday,2018-05-14,01:27,SOUTHERN,"ARREST, BOOKED",500 Block of STEVENSON ST,-122.410794,37.780832,"{'type': 'Point', 'coordinates': [-122.4107939...",1.803580e+13
85,180358028.0,BURGLARY,"BURGLARY, FORCIBLE ENTRY",Monday,2018-05-14,01:19,NORTHERN,NONE,100 Block of FELL ST,-122.420332,37.776208,"{'type': 'Point', 'coordinates': [-122.4203320...",1.803580e+13
86,180357989.0,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Monday,2018-05-14,01:03,SOUTHERN,"ARREST, BOOKED",MARKET ST / 6TH ST,-122.410293,37.782231,"{'type': 'Point', 'coordinates': [-122.4102935...",1.803580e+13
87,180357989.0,OTHER OFFENSES,RECKLESS DRIVING,Monday,2018-05-14,01:03,SOUTHERN,"ARREST, BOOKED",MARKET ST / 6TH ST,-122.410293,37.782231,"{'type': 'Point', 'coordinates': [-122.4102935...",1.803580e+13


In [29]:
# Filter for 'ROBBERY' category
robbery_data = last_7_days[last_7_days['Category'] == 'ROBBERY']
filtered_data = data[(data['Date'] > (max_date - timedelta(days=15))) & (data['Category'].isin(['FRAUD', 'GAMBLING']))]

In [22]:
robbery_data

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
6,180360835.0,ROBBERY,"ROBBERY, BODILY FORCE",Tuesday,2018-05-15,01:25,SOUTHERN,"ARREST, BOOKED",0 Block of 6TH ST,-122.410042,37.781954,"{'type': 'Point', 'coordinates': [-122.4100416...",1.803610e+13
12,180360603.0,ROBBERY,"ROBBERY, BODILY FORCE",Monday,2018-05-14,21:45,TENDERLOIN,"ARREST, BOOKED",700 Block of OFARRELL ST,-122.417085,37.785215,"{'type': 'Point', 'coordinates': [-122.4170853...",1.803610e+13
17,180360352.0,ROBBERY,"ROBBERY, BODILY FORCE",Monday,2018-05-14,20:01,SOUTHERN,NONE,NATOMA ST / 9TH ST,-122.413721,37.775437,"{'type': 'Point', 'coordinates': [-122.4137213...",1.803600e+13
37,180359606.0,ROBBERY,"ROBBERY OF A COMMERCIAL ESTABLISHMENT, STRONGARM",Monday,2018-05-14,16:10,SOUTHERN,"ARREST, BOOKED",900 Block of MARKET ST,-122.408421,37.783570,"{'type': 'Point', 'coordinates': [-122.4084211...",1.803600e+13


In [23]:
# Process the 'Location' column to extract latitude and longitude
def extract_coordinates(location_str):
    location_dict = ast.literal_eval(location_str)
    return location_dict['coordinates'][1], location_dict['coordinates'][0]

In [24]:
robbery_data[['Latitude', 'Longitude']] = robbery_data['Location'].apply(lambda x: pd.Series(extract_coordinates(x)))

<ipython-input-24-f66375197484>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  robbery_data[['Latitude', 'Longitude']] = robbery_data['Location'].apply(lambda x: pd.Series(extract_coordinates(x)))
<ipython-input-24-f66375197484>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  robbery_data[['Latitude', 'Longitude']] = robbery_data['Location'].apply(lambda x: pd.Series(extract_coordinates(x)))


In [25]:
# Create a Folium map centered around San Francisco
sf_map = folium.Map(location=[37.7749, -122.4194], zoom_start=12)

In [26]:
# Add markers for robbery incidents
for _, row in robbery_data.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"Robbery on {row['Date']}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(sf_map)

In [27]:
sf_map

In [30]:
# Function to extract latitude and longitude from 'Location' column
def extract_lat_long(location_str):
    if isinstance(location_str, str):
        loc = eval(location_str)
        return loc['coordinates'][1], loc['coordinates'][0]
    return None, None

In [31]:
filtered_data[['Latitude', 'Longitude']] = filtered_data.apply(lambda row: pd.Series(extract_lat_long(row['Location'])), axis=1)

<ipython-input-31-e4154ad02957>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data[['Latitude', 'Longitude']] = filtered_data.apply(lambda row: pd.Series(extract_lat_long(row['Location'])), axis=1)
<ipython-input-31-e4154ad02957>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data[['Latitude', 'Longitude']] = filtered_data.apply(lambda row: pd.Series(extract_lat_long(row['Location'])), axis=1)


In [34]:
# Adding markers for the filtered data
for _, row in filtered_data.iterrows():
    if pd.notna(row['Latitude']) and pd.notna(row['Longitude']):
        folium.Marker(
            [row['Latitude'], row['Longitude']],
            popup=f"{row['Descript']}",
            icon=folium.Icon(icon='usd', prefix='fa')  # FontAwesome icon for fraud and gambling
        ).add_to(sf_map)

In [35]:
sf_map

In [36]:
# Saving the map to an HTML file
output_file = '/content/sf_crime_map.html'
sf_map.save(output_file)

In [37]:
# Adding markers for the filtered data
for _, row in filtered_data.iterrows():
    if pd.notna(row['Latitude']) and pd.notna(row['Longitude']):
        icon_color = 'blue' if row['Category'] == 'FRAUD' else 'green'
        folium.Marker(
            [row['Latitude'], row['Longitude']],
            popup=f"{row['Category']}: {row['Descript']}",
            icon=folium.Icon(color=icon_color, icon='usd', prefix='fa')
        ).add_to(sf_map)

In [38]:
sf_map